In [ ]:
import numpy as np 
import pandas as pd 
df = pd.read_csv('../input/european-cup2/International football results from 1872 to 2021.csv')

In [ ]:
#把所有欧洲杯比赛挑出来（不包括预选赛）
df_UEFA_all = df[df['tournament'].str.contains('UEFA', regex=True)]
df_UEFA = df_UEFA_all[df_UEFA_all['tournament']=='UEFA Euro']
df_UEFA

In [ ]:
#要求的11个国家
countries = ['Belgium','England','France','Germany','Italy','Netherlands','Poland','Portugal','Scotland','Spain','Switzerland']
countries

In [ ]:
#把322场欧洲杯比赛里涉及到上述11个国家的场次挑出来
home_team = df_UEFA.home_team.isin(countries)
away_team = df_UEFA.away_team.isin(countries)
new_df = df[home_team & away_team & df.home_score.notnull() & df.away_score.notnull()]
new_df

In [ ]:
#分析胜负场
#我也不知道为什么会红但是好像不影响过程（迷惑。jpg）
new_df.loc[:,'date'] = pd.to_datetime(new_df.loc[:,'date'])
new_df['year'] = new_df['date'].dt.year
new_df['diff_score'] = new_df['home_score']-new_df['away_score']
new_df['win_team'] = ''
new_df['diff_score'] = pd.to_numeric(new_df['diff_score'])
new_df.loc[new_df['diff_score']> 0, 'win_team'] = new_df.loc[new_df['diff_score']> 0, 'home_team']
new_df.loc[new_df['diff_score']< 0, 'win_team'] = new_df.loc[new_df['diff_score']< 0, 'away_team']
new_df.loc[new_df['diff_score']== 0, 'win_team'] = 'Draw'

new_df

In [ ]:
#获胜场数排行
s = new_df.groupby('win_team')['win_team'].count()
s.sort_values(ascending=False, inplace=True)
s.drop(labels=['Draw'], inplace=True)
s.sort_values(ascending=True,inplace=True)
s.tail(11).plot(kind='barh', figsize=(10,6), title='Rank-winners')

In [ ]:
#进球数排行
df_score_home = new_df[['home_team', 'home_score']]
column_update = ['team', 'score']
df_score_home.columns = column_update
df_score_away = new_df[['away_team', 'away_score']]
df_score_away.columns = column_update
df_score = pd.concat([df_score_home,df_score_away], ignore_index=True)
s_score = df_score.groupby('team')['score'].sum()
s_score.sort_values(ascending=False, inplace=True)
s_score.sort_values(ascending=True, inplace=True)
s_score.tail(11).plot(kind='barh', figsize=(10,6), title='Rank-Total Scores')

In [ ]:
#所以，给的11支球队中，冠军可能性最大的是法国
#但是足球是圆的，可能性和实际情况会有较大的出入
#叹气气